# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *

In [2]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files and edit datasets

LAM output analysis.

Simu irr et simu no-irr.

In [3]:
# lon_min=-10
# lon_max=4
# lat_min=34
# lat_max=46

In [4]:
noirr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/noirr'
irr_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/irr'

## LMDZ

In [ ]:
filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

lmdz_full = xr.open_mfdataset(filename)
lmdz_full = lmdz_full.rename({'time_counter':'time'})
lmdz_full.attrs['name'] = 'no_irr'
lmdz_full = lmdz_full.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))
lmdz_full

In [6]:
LMDZ_TS=True

In [ ]:
#open histmth files
if LMDZ_TS:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(noirr_dir)
    # filename = '{}/ATM/TS_MO*.nc'.format(noirr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(noirr_dir)

lmdz_noirr0 = xr.open_mfdataset(filename)
lmdz_noirr = lmdz_noirr0.rename({'time_counter':'time'})
lmdz_noirr.attrs['name'] = 'no_irr'
lmdz_noirr.attrs['plot_color']='red'
lmdz_noirr = lmdz_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_noirr['evap']=lmdz_noirr['evap'] * 3600 * 24
lmdz_noirr['evap'].attrs['units']='mm/d'
lmdz_noirr['evap'].attrs['long_name'] = 'Evapotranspiration'

lmdz_noirr['precip']=lmdz_noirr['precip'] * 3600 * 24
lmdz_noirr['precip'].attrs['long_name'] = 'Total precipitation'
lmdz_noirr['precip'].attrs['units']='mm/d'

lmdz_noirr['sens']=lmdz_noirr['sens']
lmdz_noirr['flat']=lmdz_noirr['flat']

lmdz_noirr

In [ ]:
if LMDZ_TS:
    filename = '{}/*/ATM/TS_MO/*.nc'.format(irr_dir)
else:
    filename = '{}/*/ATM/MO/*.nc'.format(irr_dir)

lmdz_irr0 = xr.open_mfdataset(filename)
lmdz_irr =lmdz_irr0.rename({'time_counter':'time'})
lmdz_irr.attrs['name'] = 'irr'
lmdz_irr.attrs['plot_color']='#0C5DA5'
lmdz_irr = lmdz_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

lmdz_irr['evap']=lmdz_irr['evap'] * 3600 * 24
lmdz_irr['evap'].attrs['units']='mm/d'
lmdz_irr['precip']=lmdz_irr['precip'] * 3600 * 24
lmdz_irr['precip'].attrs['units']='mm/d'
lmdz_irr['sens']=-lmdz_irr['sens']
lmdz_irr['flat']=-lmdz_irr['flat']

lmdz_irr

In [ ]:
if LMDZ_TS:
    # drop unused coordinates
    lmdz_noirr = lmdz_noirr.drop_vars('presnivs')
    lmdz_noirr = lmdz_noirr.drop_vars('time_centered')
    lmdz_irr = lmdz_irr.drop_vars('presnivs')
    lmdz_irr = lmdz_irr.drop_vars('time_centered')
    print('using TimeSeries outputs')

else:
    #add var t1 to lmdz datasets (first level of temp)
    lmdz_noirr['t1']    = lmdz_noirr['temp'][:,0,:,:]
    lmdz_noirr['t1'].attrs['long_name'] = 'Temperature at first level'
    lmdz_noirr['t1'].attrs['units']='K'
    
    lmdz_irr['t1']      = lmdz_irr['temp'][:,0,:,:]
    lmdz_irr['t1'].attrs['long_name'] = 'Temperature at first level'
    lmdz_irr['t1'].attrs['units']='K'
    print('using full outputs')

## ORCHIDEE

In [ ]:
filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)

orc_full = xr.open_mfdataset(filename)
orc_full = orc_full.rename({'time_counter':'time'})
orc_full.attrs['name'] = 'irr'
orc_full = orc_full.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_full['irrig_frac'] = orc_full['irrigmap_dyn']/orc_full['Areas'] * 100
orc_full['irrig_frac'].attrs['units'] = '%'

orc_full

In [ ]:
# filename = '{}/*/SRF/MO/*sechiba_history.nc'.format(noirr_dir)
filename = '{}/*/SRF/TS_MO/*.nc'.format(noirr_dir)

orc_noirr0 = xr.open_mfdataset(filename)
orc_noirr = orc_noirr0.rename({'time_counter':'time'})
orc_noirr.attrs['name'] = 'no_irr'
orc_noirr.attrs['plot_color']='red'
orc_noirr = orc_noirr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_noirr['irrig_frac'] = orc_full['irrig_frac']
orc_noirr['irrigation'] = orc_full['irrigation'][:,:,:]

orc_noirr

In [ ]:
# filename = '{}/*/SRF/MO/*history.nc'.format(irr_dir)
filename = '{}/*/SRF/TS_MO/*.nc'.format(irr_dir)

orc_irr0 = xr.open_mfdataset(filename)
orc_irr = orc_irr0.rename({'time_counter':'time'})
orc_irr.attrs['name'] = 'irr'
orc_irr.attrs['plot_color']='#0C5DA5'
orc_irr = orc_irr.sel(lon=slice(lon_min,lon_max),lat=slice(lat_min,lat_max))

orc_irr['irrigation'] = orc_full['irrigation'][:,:,:]
orc_irr['irrig_frac'] = orc_full['irrig_frac']

orc_irr

## Period selection

In [13]:
# Select months
months=[6,7,8]
# lmdz_irr =lmdz_irr.sel(time=lmdz_irr['time.month'].isin(months))
# lmdz_noirr = lmdz_noirr.sel(time=lmdz_noirr['time.month'].isin(months))
# lmdz_full = lmdz_full.sel(time=lmdz_full['time.month'].isin(months))

# orc_irr = orc_irr.sel(time=orc_irr['time.month'].isin(months))
# orc_noirr = orc_noirr.sel(time=orc_noirr['time.month'].isin(months))
# orc_full = orc_full.sel(time=orc_full['time.month'].isin(months))

## Masking

### IP continental subdomain

In [14]:
# Define masks
 
con_mask_2D_ORC= orc_full.mean(dim='time')['Contfrac']>0.95
con_mask_2D_ORC.attrs['name']='Continental mask (95%)'

ip_polygon_mask_2D=polygon_to_mask(orc_noirr.mean(dim="time"), iberian_peninsula)
ip_polygon_mask_2D.attrs['name']='Iberian Peninsula polygon mask'

#combine two masks
ip_mask = con_mask_2D_ORC * ip_polygon_mask_2D
ip_mask.attrs['name']='IPmask'

In [15]:
#ORC
# ip_orc_noirr    = orc_noirr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
# ip_orc_irr      = orc_irr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
ip_orc_noirr = apply_2Dmask_to_dataset(orc_noirr, ip_mask, dsname='no_irr')
ip_orc_irr = apply_2Dmask_to_dataset(orc_irr, ip_mask, dsname='irr')

In [16]:
#LMDZ
# ip_lmdz_noirr   = lmdz_noirr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
# ip_lmdz_irr     = lmdz_irr.where(con_mask_ORC).where(ip_mask_ORC['mask'])
ip_lmdz_noirr = apply_2Dmask_to_dataset(lmdz_noirr, ip_mask, dsname='no_irr')
ip_lmdz_irr = apply_2Dmask_to_dataset(lmdz_irr, ip_mask, dsname='irr')

In [17]:
# Full irrsims
# ip_orc_full     = apply_2Dmask_to_dataset(orc_full, ip_mask, dsname='irr')
# ip_lmdz_full    = apply_2Dmask_to_dataset(lmdz_full, ip_mask, dsname='irr')

### Irrigated areas

In [18]:
# Old masks and methods
#create a masks of data points where irrig_frac >5%
# irr_mask = (orc_full['irrigmap_dyn']/orc_full['Areas'])>0.05

#irrigated vs nonirr areas
# irr_orc_noirr=orc_noirr.where(irr_mask)
# irr_orc_irr=orc_irr.where(irr_mask)
# irr_lmdz_noirr=lmdz_noirr.where(irr_mask)
# irr_lmdz_irr=lmdz_irr.where(irr_mask)

# irr_ip_orc_noirr=ip_orc_noirr.where(irr_mask)
# irr_ip_orc_irr=ip_orc_irr.where(irr_mask)
# irr_ip_lmdz_noirr=ip_lmdz_noirr.where(irr_mask)
# irr_ip_lmdz_irr=ip_lmdz_irr.where(irr_mask)

# #non irrigated areas
# nonirr_orc_noirr=orc_noirr.where(~irr_mask)
# nonirr_orc_irr=orc_irr.where(~irr_mask)
# nonirr_lmdz_noirr=lmdz_noirr.where(~irr_mask)
# nonirr_lmdz_irr=lmdz_irr.where(~irr_mask)

# nonirr_ip_orc_noirr=ip_orc_noirr.where(~irr_mask)
# nonirr_ip_orc_irr=ip_orc_irr.where(~irr_mask)
# nonirr_ip_lmdz_noirr=ip_lmdz_noirr.where(~irr_mask)
# nonirr_ip_lmdz_irr=ip_lmdz_irr.where(~irr_mask)

#for both full datasets
# ip_orc_full=orc_full.where(ip_mask_ORC['mask']).where(con_mask_ORC)
# ip_lmdz_full=lmdz_full.where(ip_mask_ORC['mask']).where(con_mask_ORC)
# irr_ip_orc_full=ip_orc_full.where(irr_mask)
# nonirr_ip_orc_full=ip_orc_full.where(~irr_mask)

In [19]:
# Define 2D masks

# Applied irrigation
irr_mask_low    = ip_orc_irr.mean(dim='time')['irrigation']<0.05
irr_mask_high   = ip_orc_irr.mean(dim='time')['irrigation']>0.1
irr_mask_med    = (~irr_mask_low) * (~irr_mask_high)

In [20]:
#irrigated fraction
# irr_mask5   = ip_orc_full.mean(dim='time')['irrig_frac']<5
# irr_mask10  = ip_orc_full.mean(dim='time')['irrig_frac']<10
# irr_mask15  = ip_orc_full.mean(dim='time')['irrig_frac']<15
# irr_mask20  = ip_orc_full.mean(dim='time')['irrig_frac']<20
# irr_mask30  = ip_orc_full.mean(dim='time')['irrig_frac']<30
# irr_mask40  = ip_orc_full.mean(dim='time')['irrig_frac']<40

In [21]:
# Define datasets
lowirr_ip_orc_noirr = apply_2Dmask_to_dataset(ip_orc_noirr, irr_mask_low, dsname='lowirr_noirr')
lowirr_ip_orc_irr = apply_2Dmask_to_dataset(ip_orc_irr, irr_mask_low, dsname='lowirr_irr')

medirr_ip_orc_noirr = apply_2Dmask_to_dataset(ip_orc_noirr, irr_mask_med, dsname='medirr_noirr')
medirr_ip_orc_irr = apply_2Dmask_to_dataset(ip_orc_irr, irr_mask_med, dsname='medirr_irr')

highirr_ip_orc_noirr = apply_2Dmask_to_dataset(ip_orc_noirr, irr_mask_high, dsname='highirr_noirr')
highirr_ip_orc_irr = apply_2Dmask_to_dataset(ip_orc_irr, irr_mask_high, dsname='highirr_irr')

## Mean, difference, and (spatially) aggregated datasets

In [22]:
# ip_orc_noirr_mean=ip_orc_noirr.mean(dim='time')
# ip_orc_noirr_mean.attrs['name']='IP ORC no_irr'
# ip_orc_irr_mean=ip_orc_irr.mean(dim='time')
# ip_orc_irr_mean.attrs['name']='IP ORC irr'
# ip_lmdz_noirr_mean=ip_lmdz_noirr.mean(dim='time')
# ip_lmdz_noirr_mean.attrs['name']='IP LMDZ no_irr'
# ip_lmdz_irr_mean=ip_lmdz_irr.mean(dim='time')
# ip_lmdz_irr_mean.attrs['name']='IP LMDZ irr'

# orc_full_mean=orc_full.mean(dim='time')
# orc_full_mean.attrs['name']='ORC irr'
# lmdz_full_mean=lmdz_full.mean(dim='time')
# lmdz_full_mean.attrs['name']='LMDZ irr'

# ip_orc_full_mean=ip_orc_full.mean(dim='time')
# ip_orc_full_mean.attrs['name']='IP ORC irr'
# ip_lmdz_full_mean=ip_lmdz_full.mean(dim='time')
# ip_lmdz_full_mean.attrs['name']='IP LMDZ irr'

In [23]:
#old aggr datasets
# # ip_orc_full_aggr = ip_orc_full.mean(dim='lon').mean(dim='lat')
# ip_lmdz_full_aggr = ip_lmdz_full.mean(dim='lon').mean(dim='lat')
# ip_orc_irr_aggr = ip_orc_irr.mean(dim='lon').mean(dim='lat')
# ip_lmdz_irr_aggr = ip_lmdz_irr.mean(dim='lon').mean(dim='lat')
# ip_orc_noirr_aggr = ip_orc_noirr.mean(dim='lon').mean(dim='lat')
# ip_lmdz_noirr_aggr = ip_lmdz_noirr.mean(dim='lon').mean(dim='lat')
# ip_orc_diff_aggr = ip_orc_diff.mean(dim='lon').mean(dim='lat')
# ip_lmdz_diff_aggr = ip_lmdz_diff.mean(dim='lon').mean(dim='lat')

# irr_ip_orc_full_aggr = irr_ip_orc_full.mean(dim='lon').mean(dim='lat')
# irr_ip_orc_irr_aggr = irr_ip_orc_irr.mean(dim='lon').mean(dim='lat')
# irr_ip_lmdz_irr_aggr = irr_ip_lmdz_irr.mean(dim='lon').mean(dim='lat')
# irr_ip_orc_noirr_aggr = irr_ip_orc_noirr.mean(dim='lon').mean(dim='lat')
# irr_ip_lmdz_noirr_aggr = irr_ip_lmdz_noirr.mean(dim='lon').mean(dim='lat')
# nonirr_ip_orc_full_aggr = nonirr_ip_orc_full.mean(dim='lon').mean(dim='lat')

#name new datasets
# ip_orc_full_aggr.attrs['name']='ORC irr IP mean'
# ip_lmdz_full_aggr.attrs['name']='LMDZ irr IP mean'
# ip_orc_irr_aggr.attrs['name']='ORC irr IP mean'
# ip_lmdz_irr_aggr.attrs['name']='LMDZ irr IP mean'
# ip_orc_noirr_aggr.attrs['name']='ORC no_irr IP mean'
# ip_lmdz_noirr_aggr.attrs['name']='LMDZ no_irr IP mean'
# ip_orc_diff_aggr.attrs['name']='ORC diff IP mean'
# ip_lmdz_diff_aggr.attrs['name']='LMDZ diff IP mean'

# irr_ip_orc_full_aggr.attrs['name']='ORC_irr IP irrZone mean'
# irr_ip_orc_irr_aggr.attrs['name']='ORC_irr IP irrZone mean'
# irr_ip_lmdz_irr_aggr.attrs['name']='LMDZ_irr IP irrZone mean'
# irr_ip_orc_noirr_aggr.attrs['name']='ORC_no_irr IP irrZone mean'
# irr_ip_lmdz_noirr_aggr.attrs['name']='LMDZ_no_irr IP irrZone mean'
# nonirr_ip_orc_full_aggr.attrs['name']='ORC_irr IP nonirrZone mean'

#add irr and nonirr distinction for diff datasets
# irr_ip_orc_diff_aggr = irr_ip_orc_diff.mean(dim='lon').mean(dim='lat')
# nonirr_ip_orc_diff_aggr = nonirr_ip_orc_diff.mean(dim='lon').mean(dim='lat')

# irr_ip_orc_diff_aggr.attrs['name']='ORC diff IP irrZone mean'
# nonirr_ip_orc_diff_aggr.attrs['name']='ORC diff IP nonirrZone mean'

In [24]:
# Temporal mean for IP sims

ip_orc_noirr_mean = mean_dataset(ip_orc_noirr)
ip_orc_irr_mean = mean_dataset(ip_orc_irr)
ip_lmdz_noirr_mean = mean_dataset(ip_lmdz_noirr)
ip_lmdz_irr_mean = mean_dataset(ip_lmdz_irr)

#Full
# ip_orc_full_mean = mean_dataset(ip_orc_full)
# ip_lmdz_full_mean = mean_dataset(ip_lmdz_full)

#by irrig class
lowirr_ip_orc_noirr_mean = mean_dataset(lowirr_ip_orc_noirr)
lowirr_ip_orc_irr_mean = mean_dataset(lowirr_ip_orc_irr)
medirr_ip_orc_noirr_mean = mean_dataset(medirr_ip_orc_noirr)
medirr_ip_orc_irr_mean = mean_dataset(medirr_ip_orc_irr)
highirr_ip_orc_noirr_mean = mean_dataset(highirr_ip_orc_noirr)
highirr_ip_orc_irr_mean = mean_dataset(highirr_ip_orc_irr)

In [25]:
#diff (with mean of irr)
#over whole IP
ip_orc_diff, ip_orc_diff_mean = diff_dataset(ip_orc_irr, ip_orc_noirr)
ip_lmdz_diff, ip_lmdz_diff_mean = diff_dataset(ip_lmdz_irr, ip_lmdz_noirr)

#by irrig class
lowirr_ip_orc_diff, lowirr_ip_orc_diff_mean     = diff_dataset(lowirr_ip_orc_irr, lowirr_ip_orc_noirr)
medirr_ip_orc_diff, medirr_ip_orc_diff_mean     = diff_dataset(medirr_ip_orc_irr, medirr_ip_orc_noirr)
highirr_ip_orc_diff, highirr_ip_orc_diff_mean   = diff_dataset(highirr_ip_orc_irr, highirr_ip_orc_noirr)

In [26]:
#spatial aggregation over domain or subdomain
#Whole IP domain
ip_orc_noirr_aggr = aggr_dataset(ip_orc_noirr)
ip_orc_irr_aggr = aggr_dataset(ip_orc_irr)
ip_lmdz_noirr_aggr = aggr_dataset(ip_lmdz_noirr)
ip_lmdz_irr_aggr = aggr_dataset(ip_lmdz_irr)

ip_orc_diff_aggr = aggr_dataset(ip_orc_diff)
ip_lmdz_diff_aggr = aggr_dataset(ip_lmdz_diff)

# By irrigation class
lowirr_ip_orc_noirr_aggr = aggr_dataset(lowirr_ip_orc_noirr)
lowirr_ip_orc_irr_aggr = aggr_dataset(lowirr_ip_orc_irr)
lowirr_ip_orc_diff_aggr = aggr_dataset(lowirr_ip_orc_diff)

medirr_ip_orc_noirr_aggr = aggr_dataset(medirr_ip_orc_noirr)
medirr_ip_orc_irr_aggr = aggr_dataset(medirr_ip_orc_irr)
medirr_ip_orc_diff_aggr = aggr_dataset(medirr_ip_orc_diff)

highirr_ip_orc_noirr_aggr = aggr_dataset(highirr_ip_orc_noirr)
highirr_ip_orc_irr_aggr = aggr_dataset(highirr_ip_orc_irr)
highirr_ip_orc_diff_aggr = aggr_dataset(highirr_ip_orc_diff)

# Maps

In [ ]:
var='evap'
ds=ip_orc_diff
vmin=-0.0
vmax= 1.5
vmin,vmax = None, None
cmap=wet
map_ave(ds, var, vmin=vmin, vmax=vmax, cmap=cmap)

In [ ]:
var='evap'
ds=ip_orc_diff_mean
vmin=-0.0
vmax= 1.5
vmin,vmax = None, None
cmap=wet
map_plotvar(ds[var], vmin=vmin, vmax=vmax, cmap=cmap)

In [100]:
#Diff map
var='runoff'
ds1=ip_orc_irr
ds2=ip_orc_noirr
vmax= 0.3
vmin=-0.3
# vmin,vmax = None, None
cmap=emb_neutral
pvalue=0.05
sig_method=1
check_norm=False
sig_viz=3

In [ ]:
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=False)

In [ ]:
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=True, sig_method=sig_method, sig_viz=sig_viz, check_norm=check_norm, pvalue=pvalue, hatch='xxx')

In [ ]:
map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, 
             sig=True, sig_method=sig_method, sig_viz=0, pvalue=pvalue)

In [ ]:
brr

In [ ]:
#ttest pvalue plot
var='sens'
max_value=0.2
min_value=0
min_value, max_value= None, None
ds1=lmdz_irr
ds2=lmdz_noirr
p_values = xr.apply_ufunc(
    lambda x, y: ttest_ind(x, y, axis=0, nan_policy='omit').pvalue, 
    ds1[var], ds2[var],
    input_core_dims=[['time'], ['time']],
    output_core_dims=[[]],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    dask_gufunc_kwargs={'allow_rechunk': True}
)
title='pvalues for {}, JJA'.format(var)
map_plotvar(p_values, vmin=min_value, vmax=max_value, cmap=wet,title=title)

In [ ]:
# Display 2 maps of var
var = 'sens'
ds1 = lmdz_irr
ds2 = lmdz_noirr
color_map=reds

map_two_ds(ds1, ds2, var, in_cmap=color_map)

In [ ]:
#Relative diff map
var='sfcWind'
maxvalue=None
ds1=lmdz_irr
ds2=lmdz_noirr

map_rel_diff_ave(ds1, ds2, var, in_vmax=maxvalue)

In [ ]:
#Display map of var for ds on restricted area
var='t2m'
ds = lmdz_noirr
# ds = std
latmin=36.0
latmax=44
lonmin=-10
lonmax=3.5

#restrict ds to latmin, latmax, lonmin, lonmax
ds = ds.where(ds.lat >= latmin, drop=True).where(ds.lat <= latmax, drop=True)
ds = ds.where(ds.lon >= lonmin, drop=True).where(ds.lon <= lonmax, drop=True)
plotvar = ds[var].mean(dim='time')

map_plotvar(plotvar, in_cmap=emb_neutral, in_vmax=None, title=var)

# Time series and seasonnal cycle

In [ ]:
var='irrigation'

ds1=lowirr_ip_orc_irr
ds2=medirr_ip_orc_irr
ds3=highirr_ip_orc_irr
ds_list=[ds1, ds2, ds3]
# ds_list=[ds1]

year_max=2022
time_series_ave(ds_list, var, year_max=year_max, title='{} ({})'.format(var, ds1[var].attrs['units']))
seasonal_cycle_ave(ds_list, var, year_max=year_max, title='Seasonal cycle of {} ({})'.format(var, ds1[var].attrs['units']))

# Scatter plots

## By grid cell (local recycling)

In [ ]:
var1='irrigation'
ds1=irr_ip_orc_full

var2='runoff'
ds2=irr_ip_orc_diff

title='Annual mean (1 data pt = 1 grid cell)'
# title=None

# scatter_vars(ds1, ds2, var1, var2, reg=True, plot_one=True, title=title)
scatter_annual_mean(ds1, ds2, var1, var2, reg=True, plot_one=True)

In [ ]:
var1='evap'
ds1=ip_lmdz_diff_mean

var2='rain'
ds2=ip_orc_diff_mean

scatter_vars_density(ds1, ds2, var1, var2, reg=True, plot_one=False)

## Spatial aggregation (regional recycling on a given month)

In [ ]:
var1='irrigation'
ds1=ip_orc_full_aggr

var2='s_pblh'
# ds2=ip_orc_diff_aggr
ds2=irr_ip_lmdz_diff_aggr
seasons=['MAM', 'JJA', 'SON']
scatter_vars_seasons(ds1, ds2, var1, var2, reg=True, plot_one=False, coloring=True,
                      is_1D=True, seasons_list=seasons)

In [ ]:
var1='irrigation'
ds1=ip_orc_full_aggr

var2='evap'
ds2=irr_ip_orc_diff_aggr

title='Irrigated areas domain mean (1 data pt = 1 sim month)'
title=None

seasons=['JJA']
months=[6,7,8]
scatter_vars_months(ds1, ds2, var1, var2, reg=True, plot_one=True, is_1D=True, months_list=months, coloring=True, title=title)

# Irrigation class subdatasets

In [36]:
#write a function that takes several datasets and computes the average over the domain for a list of vars and return all means as pd.DataFrame
def compute_describe(ds_list, vars):
    out=pd.DataFrame()
    for ds in ds_list:
        ds_name=ds.attrs['name']
        for var in vars:
            #add a line in the dataframe
            out.loc[ds_name, var]=ds[var].mean(dim='lon').mean(dim='lat').values
            #add a 
    return out

In [ ]:
vars=['irrigation']
ds_list=[ lowirr_ip_orc_irr_mean, 
          medirr_ip_orc_irr_mean, 
         highirr_ip_orc_irr_mean]

# irrig_by_zone = compute_means(ds_list, vars)
irrig_by_zone = build_stats_df(ds_list, vars)
rename_dict={
            "lowirr_irr":"lowirr",
            "medirr_irr":"medirr",
            "highirr_irr":"highirr"
            }
irrig_by_zone.rename(index=rename_dict, inplace=True)
irrig_by_zone

In [ ]:
vars=['evap', 'rain', 'tair', 'runoff', 'mrro']
ds_list=[ lowirr_ip_orc_diff_mean,
          medirr_ip_orc_diff_mean, 
         highirr_ip_orc_diff_mean]

diff_by_zone = build_stats_df(ds_list, vars)
#change name of rows
rename_dict={
            "lowirr_irr":"lowirr",
            "medirr_irr":"medirr",
            "highirr_irr":"highirr"
            }
diff_by_zone.rename(index=rename_dict, inplace=True)
diff_by_zone

In [66]:
irrig_by_zone.attrs['name']='Irrigation by zone'
diff_by_zone.attrs['name']='Diff by zone'

In [80]:
#add all subcolumns of irrigation to diff_by_zone
# irrig_by_zone
diff_by_zone_withirr = pd.concat([diff_by_zone, irrig_by_zone], axis=1)

#export to csv
diff_by_zone_withirr.to_csv('figures/diff_by_zone.csv')

In [ ]:
#remove irrigation from diff_by_zone
diff_by_zone = diff_by_zone.drop(columns=['irrigation'])
diff_by_zone

In [ ]:
diff_by_zone_withirr["irrigation"]

In [ ]:
#scatter plot diff of evap on y axis with irrigation on x axis for all 3 diff datasets
var1=diff_by_zone_withirr["irrigation"]['mean']
var2=diff_by_zone_withirr["evap"]['mean']
var3=diff_by_zone_withirr["rain"]['mean']
var4=diff_by_zone_withirr["tair"]['mean']
title='Evap diff vs irrigation for different irrigation zones'
xmin=0
xmax=0.16
ymin=0
ymax=0.02
# plt.scatter(var1, var2)
plt.scatter(var1, var2, vmin=ymin, vmax=ymax)
# plt.scatter(var1, var4)
#plot 1:1 line
# plt.plot([0, 0.2], [0, 0.2], color='black', linestyle='--')

In [ ]:
ds1=irrig_by_zone
ds2=diff_by_zone
var1='irrigation'
var2='tair'
vmin=0
vmax=0.2
scatter_vars(ds1, ds2, var1, var2, reg=False, plot_one=False, title=title)